In [3]:
from sklearn.datasets import make_classification
from collections import Counter
import numpy as np
import pandas as pd
from matplotlib import pyplot

from numpy import mean
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from numpy import mean

from imblearn.ensemble import EasyEnsembleClassifier

In [4]:
raw_df = pd.read_csv('day_approach_maskedID_timeseries.csv')

convert_dict = {'nr. sessions': int } #convert nr.sessions to int 
raw_df = raw_df.astype(convert_dict)
raw_df.head()

,nr. sessions,total km,km Z3-4,km Z5-T1-T2,km sprinting,strength training,hours alternative,perceived exertion,perceived trainingSuccess,perceived recovery,...,km Z5-T1-T2.6,km sprinting.6,strength training.6,hours alternative.6,perceived exertion.6,perceived trainingSuccess.6,perceived recovery.6,Athlete ID,injury,Date
0,1,5.8,0.0,0.6,1.2,0.0,0.00,0.11,0.00,0.18,...,0.0,0.0,0.0,1.0,0.10,0.00,0.15,0,0,0
1,0,0.0,0.0,0.0,0.0,0.0,0.00,-0.01,-0.01,-0.01,...,0.5,1.2,0.0,0.0,0.10,0.00,0.17,0,0,1
2,1,0.0,0.0,0.0,0.0,1.0,0.00,0.10,0.00,0.17,...,0.0,0.0,0.0,0.0,-0.01,-0.01,-0.01,0,0,2
3,0,0.0,0.0,0.0,0.0,0.0,0.00,-0.01,-0.01,-0.01,...,0.0,0.0,1.0,0.0,0.10,0.00,0.17,0,0,3
4,1,0.0,0.0,0.0,0.0,0.0,1.08,0.08,0.00,0.18,...,0.0,0.0,0.0,0.0,0.11,0.00,0.17,0,0,4


In [5]:
neg, pos = np.bincount(raw_df['injury'])
total = neg + pos
print('Examples:\n    Total: {}\n    Positive: {} ({:.2f}% of total)\n'.format(
    total, pos, 100 * pos / total))

Examples:
    Total: 42766
    Positive: 583 (1.36% of total)



In [6]:
cleaned_df = raw_df.copy()

# You don't want the `Time` column.
cleaned_df.pop('Date')
cleaned_df.pop('Athlete ID')

0         0
1         0
2         0
3         0
4         0
         ..
42761    71
42762    71
42763    71
42764    71
42765    71
Name: Athlete ID, Length: 42766, dtype: int64

In [7]:
cleaned_df.head()

,nr. sessions,total km,km Z3-4,km Z5-T1-T2,km sprinting,strength training,hours alternative,perceived exertion,perceived trainingSuccess,perceived recovery,...,total km.6,km Z3-4.6,km Z5-T1-T2.6,km sprinting.6,strength training.6,hours alternative.6,perceived exertion.6,perceived trainingSuccess.6,perceived recovery.6,injury
0,1,5.8,0.0,0.6,1.2,0.0,0.00,0.11,0.00,0.18,...,0.0,0.0,0.0,0.0,0.0,1.0,0.10,0.00,0.15,0
1,0,0.0,0.0,0.0,0.0,0.0,0.00,-0.01,-0.01,-0.01,...,5.2,0.0,0.5,1.2,0.0,0.0,0.10,0.00,0.17,0
2,1,0.0,0.0,0.0,0.0,1.0,0.00,0.10,0.00,0.17,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.01,-0.01,-0.01,0
3,0,0.0,0.0,0.0,0.0,0.0,0.00,-0.01,-0.01,-0.01,...,0.0,0.0,0.0,0.0,1.0,0.0,0.10,0.00,0.17,0
4,1,0.0,0.0,0.0,0.0,0.0,1.08,0.08,0.00,0.18,...,17.6,7.2,0.0,0.0,0.0,0.0,0.11,0.00,0.17,0


In [8]:
y = np.array(cleaned_df['injury'])
X = np.array(cleaned_df.drop('injury', axis=1))

# define dataset
X, y = make_classification(n_samples=10000, n_features=2, n_redundant=0,
                           n_clusters_per_class=2, weights=[0.99], flip_y=0, random_state=7)

In [9]:
X

array([[ 1.00e+00,  5.80e+00,  0.00e+00, ...,  1.00e-01,  0.00e+00,
         1.50e-01],
       [ 0.00e+00,  0.00e+00,  0.00e+00, ...,  1.00e-01,  0.00e+00,
         1.70e-01],
       [ 1.00e+00,  0.00e+00,  0.00e+00, ..., -1.00e-02, -1.00e-02,
        -1.00e-02],
       ...,
       [ 1.00e+00,  1.00e+01,  0.00e+00, ..., -1.00e-02, -1.00e-02,
        -1.00e-02],
       [ 1.00e+00,  1.51e+01,  0.00e+00, ...,  7.90e-01,  8.00e-01,
         1.10e-01],
       [ 1.00e+00,  1.22e+01,  0.00e+00, ...,  5.10e-01,  8.30e-01,
         1.50e-01]])

In [10]:
y

array([0, 0, 0, ..., 1, 1, 1])

In [11]:
# summarize class distribution
counter = Counter(y)
print(counter)

Counter({0: 42183, 1: 583})


In [18]:
# easy ensemble for imbalanced classification

model = EasyEnsembleClassifier(n_estimators=1000)
# define evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
scores = cross_val_score(model, X, y, scoring='roc_auc', cv=cv, n_jobs=-1)
# summarize performance
print('Mean ROC AUC: %.3f' % mean(scores))

Mean ROC AUC: 0.640
